In [1]:
# filelist
import cv2
import numpy as np
import os
from pathlib import Path
root = os.getcwd() # Don't change this
data_dirname = '/data_test/plantvillage/' # Change as you like 
p = Path(root + data_dirname)
p.mkdir(exist_ok=True) # should "/content/gdrive/My Drive/data"

classes = ['Apple___Apple_scab','Apple___healthy']
for c in classes:
  print(c)
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))
# Pick first image of first class (i.e., cat)
filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]

Apple___Apple_scab
Apple___healthy


In [2]:
from keras.applications.mobilenet import MobileNet
conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=(40, 40, 3))
type(conv_base)

keras.engine.functional.Functional

In [10]:
from keras.layers import Dense,GlobalAveragePooling2D, Concatenate, Input, Flatten, Lambda
from keras.models import Model


number_of_tiles = 9
image_net = conv_base
tiles = Input((9,40,40,3))
tile1 = Lambda(lambda x: x[:,0,:,:,:])(tiles) 
tile2 = Lambda(lambda x: x[:,1,:,:,:])(tiles) 
tile3 = Lambda(lambda x: x[:,2,:,:,:])(tiles) 
tile4 = Lambda(lambda x: x[:,3,:,:,:])(tiles) 
tile5 = Lambda(lambda x: x[:,4,:,:,:])(tiles) 
tile6 = Lambda(lambda x: x[:,5,:,:,:])(tiles) 
tile7 = Lambda(lambda x: x[:,6,:,:,:])(tiles) 
tile8 = Lambda(lambda x: x[:,7,:,:,:])(tiles) 
tile9 = Lambda(lambda x: x[:,8,:,:,:])(tiles) 

embedding_of_tile1 = image_net(tile1) 
embedding_of_tile2 = image_net(tile2)
embedding_of_tile3 = image_net(tile3) 
embedding_of_tile4 = image_net(tile4) 
embedding_of_tile5 = image_net(tile5) 
embedding_of_tile6 = image_net(tile6)
embedding_of_tile7 = image_net(tile7)
embedding_of_tile8 = image_net(tile8)
embedding_of_tile9 = image_net(tile9)

concatonation = Concatenate(axis=1)([
    embedding_of_tile1,
    embedding_of_tile2,
    embedding_of_tile3,
    embedding_of_tile4,
    embedding_of_tile5,
    embedding_of_tile6,
    embedding_of_tile7,
    embedding_of_tile8,
    embedding_of_tile9
    ])
out = Dense(1, activation="tanh")(concatonation)
out = Flatten()(out)

# Specify model
model = Model(inputs=tiles, outputs=out)

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 9, 40, 40,   0           []                               
                                3)]                                                               
                                                                                                  
 lambda_36 (Lambda)             (None, 40, 40, 3)    0           ['input_6[0][0]']                
                                                                                                  
 lambda_37 (Lambda)             (None, 40, 40, 3)    0           ['input_6[0][0]']                
                                                                                                  
 lambda_38 (Lambda)             (None, 40, 40, 3)    0           ['input_6[0][0]']          

In [12]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:num_base_layers]:
    layer.trainable=False
for layer in model.layers[num_base_layers:]:
    layer.trainable=True

Total number of layers is 14
Number of pretrained base layers is 86


In [27]:

from isort import file
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator(x_train, 16)
validation_generator = PermNetDataGenerator(x_test, 16)


# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model - make it trainable
model.compile(optimizer=sgd_optimizer,loss=tfr.keras.losses.MeanSquaredLoss(), metrics=['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 10 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=5)

ModuleNotFoundError: No module named 'tensorflow_ranking'

In [29]:

from tensorflow_ranking import losses

ModuleNotFoundError: No module named 'tensorflow_ranking'